# Madrid, Spain

**Source of original dataset:** https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=7c2843010d9c3610VgnVCM2000001f4a900aRCRD\&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD\&vgnextfmt=default

**Location of accidents:** CALLE, NUMERO

**Date of accidents:** Date

**Outcome of accidents:** Fatality, Serious Injury, Light Injury, PDO

In [ ]:
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('display.max_colwidth', -1)
import numpy as np
from plotly import graph_objects as go
import plotly.express as px
from itertools import chain
import matplotlib.pyplot as plt
import pyproj

Setup input files

In [ ]:
data_dir = "../data/madrid/"

accident_files = ["AccidentesBicicletas_2010.csv",
                  "AccidentesBicicletas_2011.csv",
                  "AccidentesBicicletas_2012.csv",
                  "AccidentesBicicletas_2013.csv",
                  "AccidentesBicicletas_2014.csv",
                  "AccidentesBicicletas_2015.csv",
                  "AccidentesBicicletas_2016.csv",
                  "AccidentesBicicletas_2017.csv",
                  "AccidentesBicicletas_2018.csv",
                  "AccidentesBicicletas_2019.csv"]
accidents_data_files = [data_dir + s for s in accident_files]

Read original data

In [ ]:
data_aux = []

for i, accidents_data in enumerate(accidents_data_files):
    data_acc = pd.read_csv(accidents_data, sep=';', encoding="ANSI")
    data_aux.append(data_acc)


Fixing column names

In [ ]:
for i in range(len(data_aux)):
    name1 = ['LUGAR ACCIDENTE',
             'Nº',
             'Tipo Vehiculo',
             'LESIVIDAD*'
            ]
    name2 = ['CALLE',
             'NÚMERO',
             'TIPO VEHÍCULO',
             'LESIVIDAD'          
             ]
    for key1, key2 in zip(name1, name2):
        try:
            data_aux[i] = data_aux[i].rename(columns={key1: key2})
        except:
            pass

In [ ]:
list_of_dfs = data_aux
list_of_dicts = [cur_df.T.to_dict().values() for cur_df in list_of_dfs]    
data = pd.DataFrame(list(chain(*list_of_dicts)))    

Create Datetime column

In [ ]:
data['Date'] = pd.to_datetime(data['FECHA'])

Setup outcomes

In [ ]:
data['LESIVIDAD'] = data['LESIVIDAD'].str.rstrip()

data['fatalities'] = 0
data.loc[(data['LESIVIDAD']=='M') | (data['LESIVIDAD']==4), 'fatalities'] = 1

data['serious_injury'] = 0
data.loc[(data['LESIVIDAD']=='HG') | (data['LESIVIDAD']==3), 'serious_injury'] = 1

data['light_injury'] = 0
data.loc[(data['LESIVIDAD']=='HL') | (data['LESIVIDAD']==1) | (data['LESIVIDAD']==2) | (data['LESIVIDAD']==5) | (data['LESIVIDAD']==6) | (data['LESIVIDAD']==7), 'light_injury'] = 1

Setup latitude & longitude column

In [ ]:
data['Latitude'] = data['GPS LATITUDE DECIMAL']
data['Longitude'] = data['GPS LONGITUDE DECIMAL']

Setup bicycles column

In [ ]:
data_bicycles = data[(data['DIRECTIONAL ANALYSIS'] == 'COLLISION WITH BICYCLE') | (data['DIRECTIONAL ANALYSIS'] == 'COLLISION WITH BICYCLIST')]

Some key statistics

In [ ]:
print('Accidents between '+str(data['Date'].min())+' and '+str(data['Date'].max()))

total_accidents = data.shape[0]
print("There are a total of "+str(total_accidents)+" accidents.")

fatalities = data["fatalities"].sum()
print("There are a total of "+str(fatalities)+" fatalities.")

serious_injuries = data["serious_injury"].sum()
print("There are a total of "+str(serious_injuries)+" seriously injured.")

light_injuries = data["light_injury"].sum()
print("There are a total of "+str(light_injuries)+" injured.")

Bicycle accidents

In [ ]:
data_bicycles = data
data_bicycles.head()

Save to file

In [ ]:
print(data_bicycles.shape)
data_bicycles.to_csv('cycling_safety_madrid.csv')
print('Wrote file to: cycling_safety_madrid.csv')